In [81]:
import asyncio
import chess
import chess.engine
import chess.pgn
from datetime import datetime
import nest_asyncio
import pandas as pd
from stockfish import Stockfish
import requests

In [91]:
nest_asyncio.apply()

In [87]:
path = "/mnt/c/Users/x_BrownBlanC/Coding/stockfish-windows-x86-64-avx2/stockfish/stockfish-windows-x86-64-avx2.exe"
async def initialize_engine(path: str) -> chess.engine.SimpleEngine:
    # Run exe file
    _, engine =  await chess.engine.SimpleEngine.popen_uci(path)
    print("Engine Initialized")  
    return engine

In [92]:
async def chess_analysis(fens: list[str]) -> None:
    engine = await initialize_engine(path)
    print("Engine Initialized")
    
    # Perform additional tasks with engine
    for fen in fens:
        board = chess.Board(fen)
        info = await engine.analyse(board, chess.engine.Limit(time=0.1))
        print(info)
    # Clean up
    await engine.quit()
    
await chess_analysis(fens)


AttributeError: '_Local' object has no attribute '_loop'

AttributeError: '_Local' object has no attribute '_loop'

In [3]:
stockfish_path = "/mnt/c/Users/x_BrownBlanC/Coding/stockfish/stockfish-windows-x86-64-sse41-popcnt.exe"
stockfish = Stockfish(path=stockfish_path, depth=20, parameters={"Threads": 10, "Minimum Thinking Time": 0, "Hash": 4096})

In [4]:
chess_username = "CheyDB"
chess_email = "cheydb@rocketmail.com"
url = f"https://api.chess.com/pub/player/{chess_username}/games/archives"
headers = {"User-Agent": f"username:{chess_username}, email:{chess_email}"}

In [5]:
def get_monthly_archive_urls(chess_username=chess_username, url=url, headers=headers):
    response = requests.get(url=url, headers=headers)
    monthly_archive_urls = response.json()["archives"]
    return monthly_archive_urls

In [6]:
monthly_archive_urls = get_monthly_archive_urls()

In [7]:
def get_games_from_monthly_archive(monthly_archive_url):
    response = requests.get(monthly_archive_url, headers=headers)
    games = response.json()["games"]
    return games

In [8]:
games_list = []
for monthly_archive_url in monthly_archive_urls:
    games = get_games_from_monthly_archive(monthly_archive_url)
    games_list.extend(games)
print(len(games_list))
    

1355


In [9]:
def get_live_games(games):
    live_games = []
    for game in games:
        if game["url"].split("/")[-2] == "live":
            live_games.append(game)
    return live_games

In [10]:
def get_daily_games(games):
    daily_games = []
    for game in games:
        if game["url"].split("/")[-2] == "daily":
            daily_games.append(game)
    return daily_games

In [11]:
daily_games = get_daily_games(games_list)
print(len(daily_games))

314


In [12]:
live_games = get_live_games(games_list)
print(len(live_games))

1041


In [13]:
def extract_pgn(games_list):
    pgn_list = []
    for game in games_list:
        pgn_list.append(game["pgn"])
    return pgn_list

In [14]:
extracted_pgn = extract_pgn(live_games)

In [42]:
print(__name__)

__main__


In [37]:
def parse_pgn(pgn_text_list):
    game_data = []
    move_data = []
    for pgn_text in pgn_text_list:
        pgn = io.StringIO(pgn_text)
        game = chess.pgn.read_game(pgn)

    # Extract game-level data
        game_data.append({
            "Game ID": game.headers["Link"].split("/")[-1],
            "Date": game.headers.get("Date"),
            "Start Time": game.headers.get("UTCTime"),
            "End Time": game.headers.get("EndTime"),
            "White Player": game.headers.get("White"),
            "Black Player": game.headers.get("Black"),
            "White Player ELO(After game)": game.headers.get("WhiteElo"),
            "Black Player ELO(After game)": game.headers.get("BlackElo"),
            "Final Position(FEN)": game.headers.get("CurrentPosition"),
            "Opening Name": game.headers.get("ECOUrl").split("/")[-1] if game.headers.get("ECOUrl") else None,
            "ECO": game.headers.get("ECO"),
            "ECO url": game.headers.get("ECOUrl"),
            "Time Control": game.headers.get("TimeControl"),
            "Winner": game.headers.get("Result"),
            "I Won": True if game.headers.get("White") == "CheyDB" and game.headers.get("Result") == "1-0" or game.headers.get("Black") == "CheyDB" and game.headers.get("Result") == "0-1" else False,
            "Result": game.headers.get("Result"),
            "Termination": game.headers.get("Termination"),
            "Total Time Spent": str(datetime.strptime(game.headers.get("EndTime"), '%H:%M:%S') - datetime.strptime(game.headers.get("UTCTime"), '%H:%M:%S')),
            "Game Link": game.headers.get("Link")
        })
        
        
        # Extract move-level data
        move_number = 1
        board = chess.Board()
        for move in game.mainline_moves():
            # best_move = stockfish.get_best_move() if move_number > 1 else None
            san = board.san(move)
            is_capture = board.is_capture(move)
            board.push(move)
            fen_after = board.fen()
            # stockfish_fen_position = stockfish.set_fen_position(fen_after)
            # board_eval_after_move = stockfish.get_evaluation()

            move_data.append({
                "Game ID": game.headers["Link"].split("/")[-1],
                "Move Number": (move_number),
                "Move": move.uci(),
                "San": san,
                "Move Colour": "White" if board.turn == chess.BLACK else "Black",  # Color is based on turn after move
                "Piece Moved": board.piece_at(move.to_square),
                "Starting Square": chess.square_name(move.from_square),
                "Ending Square": chess.square_name(move.to_square),
                "FEN After Move": fen_after,
                # "Best Move": best_move,
                # "Board Eval After Move(type)": board_eval_after_move['type'],
                # "Board Eval After Move(type)": board_eval_after_move['value'],
                "Capture": is_capture,
                "Check": board.is_check(),
                "Checkmate": board.is_checkmate(),
                "Promotion": bool(move.promotion)
            })
        
            # Update move number
            if board.turn == chess.WHITE:
                move_number += 1

    return game_data, move_data

In [38]:
game_data, move_data = parse_pgn(extracted_pgn)

In [39]:
game_data_df = pd.DataFrame(game_data)
move_data_df = pd.DataFrame(move_data)

In [40]:
move_data_df

,Game ID,Move Number,Move,San,Move Colour,Piece Moved,Starting Square,Ending Square,FEN After Move,Capture,Check,Checkmate,Promotion
0,3023955132,1,e2e4,e4,White,P,e2,e4,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...,False,False,False,False
1,3023955132,1,a7a5,a5,Black,p,a7,a5,rnbqkbnr/1ppppppp/8/p7/4P3/8/PPPP1PPP/RNBQKBNR...,False,False,False,False
2,3023955132,2,d2d4,d4,White,P,d2,d4,rnbqkbnr/1ppppppp/8/p7/3PP3/8/PPP2PPP/RNBQKBNR...,False,False,False,False
3,3023955132,2,c7c6,c6,Black,p,c7,c6,rnbqkbnr/1p1ppppp/2p5/p7/3PP3/8/PPP2PPP/RNBQKB...,False,False,False,False
4,3023955132,3,b1c3,Nc3,White,N,b1,c3,rnbqkbnr/1p1ppppp/2p5/p7/3PP3/2N5/PPP2PPP/R1BQ...,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
61002,103806042389,76,h3d3,Rd3+,White,R,h3,d3,8/8/8/3k4/6K1/3R4/8/8 b - - 38 76,False,True,False,False
61003,103806042389,76,d5e6,Ke6,Black,k,d5,e6,8/8/4k3/8/6K1/3R4/8/8 w - - 39 77,False,False,False,False
61004,103806042389,77,g4f4,Kf4,White,K,g4,f4,8/8/4k3/8/5K2/3R4/8/8 b - - 40 77,False,False,False,False
61005,103806042389,77,e6f6,Kf6,Black,k,e6,f6,8/8/5k2/8/5K2/3R4/8/8 w - - 41 78,False,False,False,False


In [ ]:
move_data_df['Move Index'] = df.groupby('Game ID').cumcount()

In [43]:
fens = list(move_data_df['FEN After Move'])

In [54]:
fens_list = []
def get_board_eval(fens):
    stockfish_fen_positions = stockfish.set_fen_position(fens)
    board_evals = stockfish.get_evaluation()
    fens_list.append(board_evals)
    print("Move Evaluated")

In [63]:
move_data_df["new column"]

Move Evaluated


In [70]:
print(fens_list)

[{'type': 'cp', 'value': 29}, {'type': 'cp', 'value': -90}]


In [ ]:
def get_best_move(fens):
    stockfish_fen_positions = stockfish.set_fen_position(fens)
    best_moves = stockfish.get_best_move()
    return best_moves

In [41]:
game_data_df

,Game ID,Date,Start Time,End Time,White Player,Black Player,White Player ELO(After game),Black Player ELO(After game),Final Position(FEN),Opening Name,ECO,ECO url,Time Control,Winner,I Won,Result,Termination,Total Time Spent,Game Link
0,3023955132,2018.08.22,17:56:20,18:04:30,CheyDB,raghav777701,1022,924,1nbqkbnr/1p2p3/2pp1p2/2r4p/N1BPP1pN/pP4B1/P1P2...,Kings-Pawn-Opening-Ware-Defense-2.d4,B00,https://www.chess.com/openings/Kings-Pawn-Open...,600,1-0,True,1-0,CheyDB won - game abandoned,0:08:10,https://www.chess.com/game/live/3023955132
1,3023993797,2018.08.22,18:15:56,18:28:17,NicholasJWilliam,CheyDB,834,1120,2kr4/ppp5/8/4n1b1/5p2/N1P5/PP2KP2/R7 w - -,Vant-Kruijs-Opening-1...e5,A00,https://www.chess.com/openings/Vant-Kruijs-Ope...,600,0-1,True,0-1,CheyDB won - game abandoned,0:12:21,https://www.chess.com/game/live/3023993797
2,3024048195,2018.08.22,18:43:29,18:57:29,CheyDB,AnnaG2468,1293,1159,r1b2k2/pp3Q2/2pp4/6N1/4P2r/3Pb1P1/PPP4P/1K1R1R...,Three-Knights-Opening,C46,https://www.chess.com/openings/Three-Knights-O...,600,1-0,True,1-0,CheyDB won by checkmate,0:14:00,https://www.chess.com/game/live/3024048195
3,3045491799,2018.09.01,11:06:08,11:17:43,gramakp,CheyDB,1217,1158,8/p1p5/3k2p1/1p2Rn2/5B2/2PN4/PP3PPP/6K1 b - -,Italian-Game-Traxler-Knight-Sacrifice-Line,C57,https://www.chess.com/openings/Italian-Game-Tr...,600,1-0,False,1-0,gramakp won by resignation,0:11:35,https://www.chess.com/game/live/3045491799
4,3048465955,2018.09.02,18:11:09,18:27:06,manueljasso,CheyDB,1107,1053,3k1r2/3Q3p/3pB3/4pP2/1p2P3/1P1P3q/8/1K4R1 b - -,Vienna-Game-Falkbeer-Stanley-Variation,C26,https://www.chess.com/openings/Vienna-Game-Fal...,600,1-0,False,1-0,manueljasso won by checkmate,0:15:57,https://www.chess.com/game/live/3048465955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1036,102312858951,2024.02.22,03:52:06,04:01:06,CheyDB,ANUSHDESANTOS,1061,1064,r3r1R1/1p3p2/p1pq1k1p/3p1Q2/3P1n1P/2P2P1B/PP6/...,Queens-Pawn-Opening-Accelerated-London-System,D00,https://www.chess.com/openings/Queens-Pawn-Ope...,600,1-0,True,1-0,CheyDB won by checkmate,0:09:00,https://www.chess.com/game/live/102312858951
1037,102313454493,2024.02.22,04:01:31,04:07:04,7A2380,CheyDB,1084,1052,r3kb1r/pp1Q2pp/2n2p2/q3p3/2B5/2N2b2/PP3PPP/3R1...,Queens-Gambit-Accepted-Central-Variation-McDon...,D20,https://www.chess.com/openings/Queens-Gambit-A...,600,1-0,False,1-0,7A2380 won by checkmate,0:05:33,https://www.chess.com/game/live/102313454493
1038,102442969995,2024.02.23,16:02:35,16:19:13,CheyDB,ELBARAKAMOHAMED,1043,1047,8/r7/4pQ1r/2P4k/1P1PN1qp/4P1P1/P5PP/5RK1 w - -,Queens-Pawn-Opening-Accelerated-London-System,D00,https://www.chess.com/openings/Queens-Pawn-Ope...,600,0-1,False,0-1,ELBARAKAMOHAMED won on time,0:16:38,https://www.chess.com/game/live/102442969995
1039,103672317427,2024.03.08,21:32:28,21:43:58,Dtuttle828,CheyDB,1048,1054,B3k2r/3p4/p4P2/1pp5/1P6/P4NP1/3R2Kp/2q5 b k -,Scotch-Game,C45,https://www.chess.com/openings/Scotch-Game,600,0-1,True,0-1,CheyDB won by resignation,0:11:30,https://www.chess.com/game/live/103672317427
